In [2]:
import torch
import torch.nn as nn
import snntorch as snn
import torch.nn.functional as nnfunc
from torch.utils.data import DataLoader, TensorDataset
import ExpNeuron as en

В данной модели используется следующая формула для вычисления мембранного потенциала:
Я реализую численное решение для следующего уравнения: $U(t) = U_0(t)\exp(-\beta t + I(t))$

Для того, чтобы численно решить данное уравнение его нужно сначала продифференцировать.
Получим: $\frac {dU(t)}{dt} = U(t)(-\beta + \frac {dI(t)}{dt})$

$I(t)$ не дифференцируема по времени, так как она принимает дискретные значения. Я решил вместо $\frac {dI(t)}{dt}$  использовать вес $w$ этого тока.
Формула численного решения диффуравнения получается: $U(t + 1) = U(t) + U(t)(-\beta + wI(t))\Delta t$

Если $U(t) \geq threshold$, то $U_{0} = U_{min}$   


In [3]:
class Neuron(nn.Module):
	def __init__(self, beta, threshold, membrane_min, membrane_zero):
		super().__init__()
		self.lnr = nn.Linear(1, 1, bias= False)
		self.nrn = en.ExpNeuron(
			beta= beta,
			threshold= threshold,
			membrane_zero= membrane_zero,
			membrane_min= membrane_min,
			learn_beta= True,
			learn_membrane_min= True,
			learn_threshold= True)
		
	
	def forward(self, spk_input):
		mem = self.nrn.init_neuron()
		spk_outpt = []
		steps = spk_input.shape[1]

		for step in range(steps):
			curl = self.lnr(spk_input[:, step])
			spk, mem = self.nrn(curl, mem)
			spk_outpt.append(spk)

		spk_outpt = torch.stack(spk_outpt, dim = 1) 
		return spk_outpt
	
	
def gen_spike_train(lambda_, num_steps):
	spike_train = torch.tensor([1 if lambda_ > element else 0 for element in torch.rand(num_steps)], dtype=torch.float32)
	return spike_train

In [ ]:
Lambda = 0.95
steps = 100
samples = 150
test_samples = 100
eps = 5e-2
btch_sz = 5


gnrtr = torch.Generator().manual_seed(0)

# Generating data in (a, b))
a = 0.8
b = 1
if a > 0.8:
	low_lambda_samples = int(samples * 0.3)
	low_lambda_arr = torch.ones(low_lambda_samples).uniform_(0, 0.5)
	lambd_arr = torch.cat((torch.ones(samples - low_lambda_samples).uniform_(a, b), low_lambda_arr))
else:
	lambd_arr = torch.ones(samples).uniform_(a, b)

test_lambd_arr = torch.ones(test_samples).uniform_(0, 1)

test_labels = ((abs(test_lambd_arr - Lambda) <= eps ) | (test_lambd_arr > Lambda)).float().unsqueeze(1)
labels = ((abs(lambd_arr - Lambda) <= eps ) | (lambd_arr > Lambda)).float().unsqueeze(1)

test_trains = torch.stack([gen_spike_train(lambd.item(), steps) for lambd in test_lambd_arr])
trains = torch.stack([gen_spike_train(lambd.item(), steps) for lambd in lambd_arr])

# train_data, test_data = random_split(TensorDataset(trains, labels), [samples * 80 // 100, samples * 20 // 100])
train_data = TensorDataset(trains, labels)
train_dataldr = DataLoader(dataset= train_data, batch_size= btch_sz)

test_data = TensorDataset(test_trains, test_labels) 
test_dataldr = DataLoader(dataset = test_data, batch_size= 1)

In [5]:
beta = torch.tensor(0.3)
threshold = torch.tensor(0.9)
membrane_min = torch.tensor(0.4)
membrane_zero = torch.tensor(0.3)

lrng_rt = 1e-2

epochs = 20
neuron = Neuron(beta = beta, threshold = threshold, membrane_min = membrane_min, membrane_zero = membrane_zero)
optim = torch.optim.Adam(neuron.parameters(), lr = lrng_rt)

In [8]:
# BCE - функция потерь
for epoch in range(epochs):
		for trns, lbls in train_dataldr:
				optim.zero_grad()
				outputs = neuron(trns.unsqueeze(-1))
				spike_cnt = outputs.mean(dim=1)

				loss = nnfunc.binary_cross_entropy(input= spike_cnt, target= lbls, reduction= "mean")
				loss.backward()
				optim.step()

				# Ограничение weight
				neuron.lnr.weight.data.clamp_(min=0.1)

		if epoch % 5 == 0:
			print((f"Epoch {epoch}, Loss: {loss.item():.4f}, "
				f"Beta: {neuron.nrn.beta.item():.4f}, Threshold: {neuron.nrn.threshold.item():.4f}, "
				f"Membrane_min: {neuron.nrn.membrane_min.item():.4f}, Linear weight: {neuron.lnr.weight.item():.4f}"))


KeyboardInterrupt: 

In [ ]:
		
# Testing accuracy
data = []
for trn, lbl in test_dataldr:
		prediction = neuron(trn).mean(dim=1).item()
		data.append([float(prediction > 0.5), lbl.item()])
		# print((f"pred: {prediction:.2f}, predicted: {prediction > 0.5}, "
		#        f"true: {lbl}"))
data = torch.tensor(data)
true_positive = data[(data[:, 0] == 1) & (data[:, 1] == 1), 0].numel()
false_positive = data[(data[:, 0] == 1) & (data[:, 1] == 0), 0].numel()
true_negative = data[(data[:, 0] == 0) & (data[:, 1] == 0), 0].numel()
false_negative = data[(data[:, 0] == 0) & (data[:, 1] == 1), 0].numel()

# Counting metrics
if true_positive and true_negative: 
		precision = true_positive / (true_positive + false_positive)
		recall = true_positive / (true_positive + false_negative)
		accuracy = (true_positive + true_negative) / (true_positive + true_negative 
																								+ false_positive + false_negative)

		print((f"precision: {precision:.2f}, recall: {recall:.2f}, accuracy : {accuracy:.2f}"))
		print((f"\ntrue_positive: {true_positive},\n"
				f"true_negative: {true_negative},\nfalse_negative: {false_negative},\n"
				f"false_positive: {false_positive}"))
else:
		print((f"Low accuracy of the model.\ntrue_positive: {true_positive},\n"
				f"true_negative: {true_negative},\nfalse_negative: {false_negative},\n"
				f"false_positive: {false_positive}"))
	
if precision and recall:
		f1 = 2 * precision * recall / (precision + recall)
		print(f"f1 metric: {f1:.3f}")

precision: 0.85, recall: 0.92, accuracy : 0.97

true_positive: 11,
true_negative: 86,
false_negative: 1,
false_positive: 2
f1 metric: 0.880


In [ ]:
for par, val in neuron.named_parameters():
	print(par, val)

# выгрузка параметров
# torch.save(neuron.state_dict(), "neuron_weights.pth")

# для загрузки
# neuron.load_state_dict(torch.load("neuron_weights.pth"))


lnr.weight Parameter containing:
tensor([[0.9573]], requires_grad=True)
nrn.threshold Parameter containing:
tensor(0.4584, requires_grad=True)
nrn.beta Parameter containing:
tensor(0.7475, requires_grad=True)
nrn.membrane_min Parameter containing:
tensor(1.1698, requires_grad=True)
